Hello to all Kagglers! I present you a relatively simple Neural Network model that I designed for this task and got a score of ~ 9766. Good luck to everyone! :) 

In [ ]:

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices


SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)
'''
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)

train['action'] = ((train['resp'].values) > 0).astype(int)


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T




def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model


batch_size = 5000
hidden_units = [150, 150, 150]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

clf = create_mlp(
    len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

clf.fit(X_train, y_train, epochs=200, batch_size=5000)

'''


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras
from sklearn.metrics import accuracy_score
import gc
from sklearn.model_selection import train_test_split
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
train_raw = reduce_mem_usage(pd.read_csv('../input/jane-street-market-prediction/train.csv'))

In [ ]:
train = train_raw[train_raw['weight'] != 0]
train = train[train['resp'] != 0]
features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

train = train.fillna(-999)

In [ ]:
f_mean

In [ ]:
features

In [ ]:
X = train.iloc[:, train.columns.str.contains('feature')]
y = (train.loc[:, 'resp'] > 0).astype(int)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, random_state = 0)

del train
gc.collect()

In [ ]:
hidden_layers = 1 
neuron_numbers = 30 
rate_of_learning = 3e-3 


def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})


def ModelTuner1(drop_rate, hiddenN = hidden_layers, neuronsN = neuron_numbers, learnR = rate_of_learning):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[130]))
    model.add(keras.layers.BatchNormalization())
    for layers in range(hiddenN):
        model.add(keras.layers.Dense(neuronsN, activation="gelu"))
        model.add(keras.layers.Dropout(drop_rate))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation="sigmoid"))

    optim = keras.optimizers.Nadam(lr = learnR)
    model.compile(loss="binary_crossentropy", optimizer = optim, metrics = ["AUC"])
    return model


params = {
            "hiddenN": [1, 2, 3, 4, 5, 6, 7, 8, 9],
            "neuronsN": np.arange(1, 1000),
            "learnR": reciprocal(3e-4, 3e-2),
            "drop_rate": reciprocal(1e-1, 5e-1)
          }

reg_keras = keras.wrappers.scikit_learn.KerasRegressor(ModelTuner1)
rsCV = RandomizedSearchCV(reg_keras, params, n_iter=10, cv=5)

In [ ]:

drop_rate = 0.1263138694014944
hiddenN = 4
learnR = 0.01861769765675586
neuronsN = 745


def ModelTuned(drop_rate, hiddenN, neuronsN, learnR):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[130]))
    model.add(keras.layers.BatchNormalization())
    for layers in range(hiddenN):
        model.add(keras.layers.Dense(neuronsN, activation="gelu"))
        model.add(keras.layers.Dropout(drop_rate))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation="sigmoid"))

    optim = keras.optimizers.Nadam(lr = learnR)
    model.compile(loss="binary_crossentropy", optimizer = optim, metrics = ["AUC"])
    return model

model = ModelTuned(drop_rate, hiddenN, neuronsN, learnR)

model.fit(X_train, y_train, epochs=12, validation_data = (X_val, y_val), callbacks=[keras.callbacks.EarlyStopping(patience = 50)], batch_size = 4096)


In [ ]:
models = []

models.append(model)

th = 0.503


f = np.median
models = models[-3:]
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

ghjkjhg

That's it!